In [1]:
# prompt: access to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [3]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'


In [4]:
data = pd.read_csv(csv_file)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.3, random_state=42)

In [6]:
# Define the data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
# Create the training data generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 2387 validated image filenames belonging to 62 classes.


In [8]:
# Create the validation data generator (without augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 1023 validated image filenames belonging to 62 classes.


In [9]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


58889256/58889256 [==============================] - 2s 0us/step


In [10]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [11]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(62, activation='softmax')(x)

In [12]:
model = Model(inputs=base_model.input, outputs=output)

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator)


Epoch 1/20
75/75 [==============================] - 1817s 24s/step - loss: 4.1305 - accuracy: 0.0184 - val_loss: 4.0295 - val_accuracy: 0.1134
Epoch 2/20
75/75 [==============================] - 128s 2s/step - loss: 3.9752 - accuracy: 0.0817 - val_loss: 3.9002 - val_accuracy: 0.1329
Epoch 3/20
75/75 [==============================] - 127s 2s/step - loss: 3.8620 - accuracy: 0.1332 - val_loss: 3.7917 - val_accuracy: 0.1711
Epoch 4/20
75/75 [==============================] - 129s 2s/step - loss: 3.7653 - accuracy: 0.1956 - val_loss: 3.6726 - val_accuracy: 0.2864
Epoch 5/20
75/75 [==============================] - 127s 2s/step - loss: 3.6634 - accuracy: 0.2824 - val_loss: 3.5822 - val_accuracy: 0.3930
Epoch 6/20
75/75 [==============================] - 130s 2s/step - loss: 3.5716 - accuracy: 0.3092 - val_loss: 3.4841 - val_accuracy: 0.3949
Epoch 7/20
75/75 [==============================] - 128s 2s/step - loss: 3.4858 - accuracy: 0.3544 - val_loss: 3.3867 - val_accuracy: 0.3851
Epoch 8/20
